# Assignment 1 - 基于RTMPose的耳朵穴位关键点检测

By Yuming Liu

Updated: 2023/6/4

**0. Env setup**

Keep using the drive disk, the data, config, and source are located in google drive, so mound the google drive each time run this notebook

In [2]:
# Mount google drive
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [7]:
# Now enter the workspace in google drive
import os
os.chdir('/content/drive/MyDrive/workspace')
!pwd

/content/drive/MyDrive/workspace


In [ ]:
# Create virtual env if not created
# ! pip install virtualenv
# ! virtualenv colab_env_ai_camp_assignment_1



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 36.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.5/468.5 kB 58.5 MB/s eta 0:00:00
created virtual environment CPython3.10.11.final.0-64 in 9747ms
  creator CPython3Posix(dest=/content/drive/MyDrive/workspace/colab_env_ai_camp_assignment_1, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==23.1.2, setuptools==67.7.2, wheel==0.40.0
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator


In [ ]:
# !chmod 755 -R ./colab_env_ai_camp_assignment_1

# # Activate env
# !source ./colab_env_ai_camp_assignment_1/bin/activate; pip -V

pip 23.1.2 from /content/drive/MyDrive/workspace/colab_env_ai_camp_assignment_1/lib/python3.10/site-packages/pip (python 3.10)


In [ ]:
# import sys
# sys.path.append("/content/drive/MyDrive/workspace/colab_env_ai_camp_assignment_1/lib/python3.10/site-packages")

1. **Install dependencies**

In [1]:
# Upgrade numpy
# Trick for upgrading numpy, fuck ^_^
!pip install -U numpy 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# Pytorch
!pip install install torch torchvision torchaudio -f https://download.pytorch.org/whl/cu113/torch_stable.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/cu113/torch_stable.html


In [4]:
# MMCV via MIM
!pip install -U openmim
!mim install mmengine
!mim install 'mmcv==2.0.0rc3'
!mim install "mmdet>=3.0.0rc6"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.openmmlab.com/mmcv/dist/cu118/torch2.0.0/index.html
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.openmmlab.com/mmcv/dist/cu118/torch2.0.0/index.html
  Using cached mmcv-2.0.0rc3-cp310-cp310-linux_x86_64.whl
  Attempting uninstall: mmcv
    Found existing installation: mmcv 2.0.0
    Uninstalling mmcv-2.0.0:
      Successfully uninstalled mmcv-2.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mmdet 3.0.0 requires mmcv<2.1.0,>=2.0.0rc4, but you have mmcv 2.0.0rc3 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-py

In [5]:
# Other necessary packages
!pip install opencv-python pillow matplotlib seaborn tqdm pycocotools -i https://pypi.tuna.tsinghua.edu.cn/simple

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


**2. Install MMPose**

In [ ]:
!git clone https://github.com/open-mmlab/mmpose.git -b tutorial2023

Cloning into 'mmpose'...
remote: Enumerating objects: 27394, done.
remote: Counting objects: 100% (1228/1228), done.
remote: Compressing objects: 100% (643/643), done.
remote: Total 27394 (delta 660), reused 998 (delta 566), pack-reused 26166
Receiving objects: 100% (27394/27394), 30.08 MiB | 16.30 MiB/s, done.
Resolving deltas: 100% (19334/19334), done.
Updating files: 100% (1537/1537), done.


In [8]:
import os
os.chdir('mmpose')
!pwd

/content/drive/MyDrive/workspace/mmpose


In [9]:
!mim install -e .

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.openmmlab.com/mmcv/dist/cu118/torch2.0.0/index.html
Obtaining file:///content/drive/MyDrive/workspace/mmpose
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.4/286.4 kB 11.2 MB/s eta 0:00:00
  Created wheel for chumpy: filename=chumpy-0.70-py3-none-any.whl size=58268 sha256=2448162bb01680f8d8af053ac8adca8a69e73c1fcfb595cc6024b730abb8bc86
  Stored in directory: /root/.cache/pip/wheels/e0/c1/ef/29ba7be03653a29ef6f2c3e1956d6c4d8877f2b243af411db1
Successfully built chumpy
  Running setup.py develop for mmpose


**3. Prepare train data and config**

In [ ]:
import os

# checkpoint for pre-trained weights files
os.mkdir('checkpoint')

# The output directory
os.mkdir('outputs')

# test images and videoes
os.mkdir('data')

os.mkdir('data/test')

In [ ]:
# 多人图片，来源：https://www.pexels.com/zh-cn/photo/2168292/
!wget https://zihao-openmmlab.obs.cn-east-3.myhuaweicloud.com/20220610-mmpose/images/multi-person.jpeg -O data/test/multi-person.jpeg

# 单人视频-唱跳篮球 
!wget https://zihao-openmmlab.obs.myhuaweicloud.com/20220610-mmpose/videos/cxk.mp4 -P data/test

# 妈妈和女儿跳舞，经微信压缩
!wget https://zihao-openmmlab.obs.myhuaweicloud.com/20220610-mmpose/videos/mother_wx.mp4 -P data/test

# 两个女生跳舞视频，来源：https://mixkit.co/free-stock-video/two-girls-having-fun-in-a-retro-restaurant-42298/
!wget https://zihao-openmmlab.obs.cn-east-3.myhuaweicloud.com/20220610-mmpose/videos/two-girls.mp4 -O data/test/two-girls.mp4

--2023-06-04 13:22:12--  https://zihao-openmmlab.obs.cn-east-3.myhuaweicloud.com/20220610-mmpose/images/multi-person.jpeg
Resolving zihao-openmmlab.obs.cn-east-3.myhuaweicloud.com (zihao-openmmlab.obs.cn-east-3.myhuaweicloud.com)... 121.36.235.132
Connecting to zihao-openmmlab.obs.cn-east-3.myhuaweicloud.com (zihao-openmmlab.obs.cn-east-3.myhuaweicloud.com)|121.36.235.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 604280 (590K) [image/jpeg]
Saving to: ‘data/test/multi-person.jpeg’

data/test/multi-per 100%[===================>] 590.12K   402KB/s    in 1.5s    

2023-06-04 13:22:15 (402 KB/s) - ‘data/test/multi-person.jpeg’ saved [604280/604280]

--2023-06-04 13:22:15--  https://zihao-openmmlab.obs.myhuaweicloud.com/20220610-mmpose/videos/cxk.mp4
Resolving zihao-openmmlab.obs.myhuaweicloud.com (zihao-openmmlab.obs.myhuaweicloud.com)... 121.36.235.132
Connecting to zihao-openmmlab.obs.myhuaweicloud.com (zihao-openmmlab.obs.myhuaweicloud.com)|121.36.235.132|

**4. Check if succeed**

In [10]:
import torch, torchvision
print('Pytorch 版本', torch.__version__)
print('CUDA 是否可用',torch.cuda.is_available())

Pytorch 版本 2.0.1+cu118
CUDA 是否可用 False


In [11]:
import mmcv
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print('MMCV版本', mmcv.__version__)
print('CUDA版本', get_compiling_cuda_version())
print('编译器版本', get_compiler_version())

MMCV版本 2.0.0
CUDA版本 11.8
编译器版本 GCC 9.3


In [12]:
import mmpose
print('mmpose版本', mmpose.__version__)

mmpose版本 1.0.0


**5. Install MMDetection**

In [ ]:
import os
os.chdir('/content/drive/MyDrive/workspace')
!pwd

/content/drive/MyDrive/workspace


In [ ]:
!git clone https://github.com/open-mmlab/mmdetection.git -b 3.x

Cloning into 'mmdetection'...
remote: Enumerating objects: 36296, done.
remote: Counting objects: 100% (947/947), done.
remote: Compressing objects: 100% (534/534), done.
remote: Total 36296 (delta 482), reused 711 (delta 400), pack-reused 35349
Receiving objects: 100% (36296/36296), 56.73 MiB | 13.99 MiB/s, done.
Resolving deltas: 100% (25417/25417), done.
Updating files: 100% (1831/1831), done.


In [ ]:
# Enter the mmdetection directory
os.chdir('mmdetection')
!pwd

/content/drive/MyDrive/workspace/mmdetection


In [ ]:
!mim install -e .

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.openmmlab.com/mmcv/dist/cu118/torch2.0.0/index.html
Obtaining file:///content/drive/MyDrive/workspace/mmdetection
  Preparing metadata (setup.py) ... done
  Attempting uninstall: mmdet
    Found existing installation: mmdet 3.0.0
    Uninstalling mmdet-3.0.0:
      Successfully uninstalled mmdet-3.0.0
  Running setup.py develop for mmdet


**6. Prepare pre-trained data and config**

In [ ]:
import os

# 创建 checkpoint 文件夹，用于存放预训练模型权重文件
os.mkdir('checkpoint')

# 创建 outputs 文件夹，用于存放预测结果
os.mkdir('outputs')

# 创建 data 文件夹，用于存放图片和视频素材
os.mkdir('data')

**7. Check if succeed**

In [ ]:
import mmdet
print('mmdetection版本', mmdet.__version__)

mmdetection版本 3.0.0


**8. Download Dataset**

Uploaded to driver by hand

**9. Train model for object detection(mmdetection)**

In [ ]:
os.chdir('/content/drive/MyDrive/workspace/mmdetection')
!pwd

/content/drive/MyDrive/workspace/mmdetection


In [ ]:
# RTMDet Model
!python tools/train.py data/rtmdet_tiny_ear.py

06/04 14:03:14 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.10.11 (main, Apr  5 2023, 14:15:10) [GCC 9.4.0]
    CUDA available: False
    numpy_random_seed: 47479171
    GCC: x86_64-linux-gnu-gcc (Ubuntu 9.4.0-1ubuntu1~20.04.1) 9.4.0
    PyTorch: 2.0.1+cu118
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.7.3 (Git Hash 6dbeffbae1f23cbbeae17adb7b5b13f1f37c080e)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - Build settings: BLAS_INFO=mkl, BUILD_TYPE=Release, CUDA_VERSION=11.8, CUDNN_VERSION=8.7.0, CXX_COMPILER=/opt/rh/devtoolset-9/root/usr/bin/c++, CXX_FLAGS= -D_GLIBCXX_USE_CXX11_ABI=0 -fabi-version=11 -Wno-deprecated -f

In [ ]:
# Test the precision
!python tools/test.py data/rtmdet_tiny_ear.py \
                      work_dirs/rtmdet_tiny_ear/epoch_90.pth

06/05 01:14:24 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.10.11 (main, Apr  5 2023, 14:15:10) [GCC 9.4.0]
    CUDA available: False
    numpy_random_seed: 1527770135
    GCC: x86_64-linux-gnu-gcc (Ubuntu 9.4.0-1ubuntu1~20.04.1) 9.4.0
    PyTorch: 2.0.1+cu118
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.7.3 (Git Hash 6dbeffbae1f23cbbeae17adb7b5b13f1f37c080e)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - Build settings: BLAS_INFO=mkl, BUILD_TYPE=Release, CUDA_VERSION=11.8, CUDNN_VERSION=8.7.0, CXX_COMPILER=/opt/rh/devtoolset-9/root/usr/bin/c++, CXX_FLAGS= -D_GLIBCXX_USE_CXX11_ABI=0 -fabi-version=11 -Wno-deprecated 

**10. Model optimize to release (RTMDet object decection model)**

The original trained model contains a lot of train related info, which are not needed for prediction, so we need to strip these info and convert the model to a small, resized one for product use

In [ ]:
!pwd
!python tools/model_converters/publish_model.py \
        work_dirs/rtmdet_tiny_ear/epoch_90.pth \
        checkpoint/rtmdet_tiny_ear_epoch_90_202306050910.pth

/content/drive/MyDrive/workspace/mmdetection
06/05 01:10:46 - mmengine - INFO - Key `message_hub` will be removed because it is not in save_keys. If you want to keep it, please set --save-keys.
06/05 01:10:46 - mmengine - INFO - Key `optimizer` will be removed because it is not in save_keys. If you want to keep it, please set --save-keys.
06/05 01:10:46 - mmengine - INFO - Key `param_schedulers` will be removed because it is not in save_keys. If you want to keep it, please set --save-keys.
06/05 01:10:46 - mmengine - INFO - Key `ema_state_dict` will be removed because it is not in save_keys. If you want to keep it, please set --save-keys.
06/05 01:10:47 - mmengine - INFO - The published model is saved at checkpoint/rtmdet_tiny_ear_epoch_90_202306050910-4d5df3d4.pth.


In [ ]:
# Now test the prediction
# RTMDet
!python demo/image_demo.py \
        data/test_ear/ear_3.jpg \
        data/rtmdet_tiny_ear.py \
        --weights checkpoint/rtmdet_tiny_ear_epoch_90_202306050910-4d5df3d4.pth \
        --out-dir outputs/E2_rtmdet \
        --device cpu \
        --pred-score-thr 0.3

Loads checkpoint by local backend from path: checkpoint/rtmdet_tiny_ear_epoch_90_202306050910-4d5df3d4.pth
06/05 01:47:00 - mmengine - WARNING - Failed to search registry with scope "mmdet" in the "function" registry tree. As a workaround, the current "function" registry in "mmengine" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmdet" is a correct scope, or whether the registry is initialized.
06/05 01:47:00 - mmengine - WARNING - `Visualizer` backend is not initialized because save_dir is None.
/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: 
torch.meshgrid: in an upcoming release, it will be required to pass the indexing
argument. (Triggered internally at 
../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Inference ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━   
results have been saved at outputs/E2_rtmdet


**11. Train model for RTMPose**

In [13]:
os.chdir('/content/drive/MyDrive/workspace/mmpose')
!pwd

/content/drive/MyDrive/workspace/mmpose


In [13]:
# Faster R CNN
!python tools/train.py data/rtmpose-s-ear.py

06/05 05:25:17 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.10.11 (main, Apr  5 2023, 14:15:10) [GCC 9.4.0]
    CUDA available: True
    numpy_random_seed: 21
    GPU 0: Tesla T4
    CUDA_HOME: /usr/local/cuda
    NVCC: Cuda compilation tools, release 11.8, V11.8.89
    GCC: x86_64-linux-gnu-gcc (Ubuntu 9.4.0-1ubuntu1~20.04.1) 9.4.0
    PyTorch: 2.0.1+cu118
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.7.3 (Git Hash 6dbeffbae1f23cbbeae17adb7b5b13f1f37c080e)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.8
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm

In [14]:
# Test the precision
!python tools/test.py data/rtmpose-s-ear.py \
                      work_dirs/rtmpose-s-ear/epoch_190.pth

06/05 09:04:12 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.10.11 (main, Apr  5 2023, 14:15:10) [GCC 9.4.0]
    CUDA available: True
    numpy_random_seed: 21
    GPU 0: Tesla T4
    CUDA_HOME: /usr/local/cuda
    NVCC: Cuda compilation tools, release 11.8, V11.8.89
    GCC: x86_64-linux-gnu-gcc (Ubuntu 9.4.0-1ubuntu1~20.04.1) 9.4.0
    PyTorch: 2.0.1+cu118
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.7.3 (Git Hash 6dbeffbae1f23cbbeae17adb7b5b13f1f37c080e)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.8
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm

**12. Model optimize to release (RTMPose pose estimation model)**

In [31]:
!pwd
!python tools/misc/publish_model.py \
        work_dirs/rtmpose-s-ear/epoch_190.pth \
        checkpoint/rtmpose-s-ear-202306051805.pth

/content/drive/MyDrive/workspace/mmpose
06/05 10:14:52 - mmengine - INFO - Key `message_hub` will be removed because it is not in save_keys. If you want to keep it, please set --save-keys.
06/05 10:14:52 - mmengine - INFO - Key `optimizer` will be removed because it is not in save_keys. If you want to keep it, please set --save-keys.
06/05 10:14:52 - mmengine - INFO - Key `param_schedulers` will be removed because it is not in save_keys. If you want to keep it, please set --save-keys.
06/05 10:14:52 - mmengine - INFO - Key `ema_state_dict` will be removed because it is not in save_keys. If you want to keep it, please set --save-keys.


**13. Test prediction**

In [15]:
# RTMDet --> RTMPose
!python demo/topdown_demo_with_mmdet.py \
        data/rtmdet_tiny_ear.py \
        checkpoint/rtmdet_tiny_ear_epoch_90_202306050910-4d5df3d4.pth \
        data/rtmpose-s-ear.py \
        checkpoint/rtmpose-s-ear-202306051805-3078a8bc_20230605.pth \
        --input data/test_ear/ear_3.jpg \
        --output-root outputs/G2_RTMDet-RTMPose \
        --device cpu \
        --bbox-thr 0.5 \
        --kpt-thr 0.5 \
        --nms-thr 0.3 \
        --radius 36 \
        --thickness 30 \
        --draw-bbox \
        --draw-heatmap \
        --show-kpt-idx

Loads checkpoint by local backend from path: checkpoint/rtmdet_tiny_ear_epoch_90_202306050910-4d5df3d4.pth
Loads checkpoint by local backend from path: checkpoint/rtmpose-s-ear-202306051805-3078a8bc_20230605.pth
06/05 14:58:52 - mmengine - WARNING - `Visualizer` backend is not initialized because save_dir is None.
/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
/content/drive/MyDrive/workspace/mmpose/mmpose/models/heads/coord_cls_heads/rtmcc_head.py:217: UserWarning: The predicted simcc values are normalized for visualization. This may cause discrepancy between the keypoint scores and the 1D heatmaps.
  warnings.warn('The predicted simcc values are normalized for '
